In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

In [4]:
df = pd.read_csv("data/mongo_data.csv", index_col='ts')
df = df.drop('Unnamed: 0', 1)
#Remove cols with the same value

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

df = df.loc[:,np.invert(unique_cols(df))]
print(df.shape)
df.tail()

C:\Users\pedro\AppData\Local\Temp\ipykernel_35928\468073868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed: 0', 1)


(1928755, 39)


,S_SUM,U_L1_L2,I_SUM,F,U_L3_L1,C_phi_L1,U_L3_N,RealEc_L1,RealEc_L2,RealEc_L3,...,S_L3,Q_L1,S_L2,U_L2_N,S_L1,U_L1_N,I_L1,RealEc_SUM,I_L2,I_L3
ts,,,,,,,,,,,,,,,,,,,,,
2022-03-16T11:46:25.931837314Z,144890.16,408.87,1.08,49.97,407.98,0.97,235.64,44933936,43492684,43453100,...,46857.12,10264.62,50062.00,236.46,47971.05,235.85,203.40,131879832,211.72,198.85
2022-03-16T11:46:27.213046140Z,143902.53,409.29,1.12,49.97,406.56,0.99,234.81,44933952,43492700,43453120,...,47984.26,7155.61,46749.76,236.46,49168.52,235.03,209.20,131879880,197.71,204.36
2022-03-16T11:46:28.499715032Z,143457.63,408.75,1.14,49.97,405.88,0.99,234.87,44933968,43492712,43453132,...,46998.51,7665.49,47098.83,236.24,49360.28,234.77,210.25,131879920,199.37,200.11
2022-03-16T11:46:29.814130324Z,144825.58,408.98,1.09,49.97,408.18,0.97,235.72,44933988,43492732,43453148,...,46598.08,10189.86,49460.64,236.48,48766.86,235.93,206.70,131879976,209.15,197.69
2022-03-16T11:46:31.116579462Z,143771.91,409.67,1.14,49.97,406.51,0.98,235.17,44934000,43492744,43453164,...,47383.55,7984.87,47779.66,236.53,48608.71,234.99,206.85,131880016,202.00,201.49


In [13]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		if i == n_in:
			print(cols)
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [14]:
from keras.preprocessing.sequence import TimeseriesGenerator
#divide data into train and test
train_ind = int(len(df)*0.7)
cv_ind = int(len(df)*0.9)
test_ind = len(df)
train = df[:train_ind]
test = df[train_ind:cv_ind]

history_window = 2 #seconds
prediction_window = 1
values = df.head(500).values
# frame as supervised learning
reframed = series_to_supervised(values, history_window, prediction_window)
# drop columns we don't want to predict
print(reframed.head())


[            0       1     2      3       4     5       6           7   \
0          NaN     NaN   NaN    NaN     NaN   NaN     NaN         NaN   
1          NaN     NaN   NaN    NaN     NaN   NaN     NaN         NaN   
2    146233.91  409.18  1.01  50.03  406.46  0.98  235.78  25869132.0   
3    145337.03  408.08  1.24  50.03  406.35  0.99  234.44  25869150.0   
4    145837.16  408.78  0.85  50.03  408.23  0.99  235.60  25869166.0   
..         ...     ...   ...    ...     ...   ...     ...         ...   
495   69422.02  415.91  0.74  50.01  412.72  0.94  240.13  25886608.0   
496   99141.77  412.45  0.85  50.01  412.28  0.97  238.09  25886618.0   
497  100708.76  414.73  0.88  50.02  414.60  0.97  239.50  25886630.0   
498  101024.67  415.65  0.97  50.02  412.11  0.97  239.30  25886638.0   
499   99773.72  412.88  0.92  50.02  412.77  0.97  238.36  25886650.0   

             8           9   ...        29       30        31      32  \
0           NaN         NaN  ...       NaN      N

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import LayerNormalization

# design network
model = Sequential()
model.add(LayerNormalization(axis=0))
model.add(LSTM(100, input_shape=(timesteps, features)))
model.add(Dense(39))
model.compile(loss='mae', optimizer='adam')


In [20]:
# split into train and test sets
values = reframed.values
n_train_seconds = 2*24*60*60 #2 days
n_test_seconds =  3*24*60*60 #3 days
train = values[:n_train_seconds, :]
test = values[n_train_seconds:n_test_seconds, :]
# split into input and outputs
train_X, train_y = train[:, :-39], train[:, -39:]
test_X, test_y = test[:, :-39], test[:, -39:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(172800, 1, 195) (172800, 39) (86400, 1, 195) (86400, 39)


In [21]:
history = model.fit(train_X, train_y, epochs=200, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)


Epoch 1/2
2400/2400 - 8s - loss: 12986233.0000 - val_loss: 13868734.0000 - 8s/epoch - 3ms/step
Epoch 2/2
2400/2400 - 6s - loss: 12986086.0000 - val_loss: 13868604.0000 - 6s/epoch - 3ms/step


In [25]:
history_results = pd.DataFrame([history.history['loss'], history.history['val_loss']], columns=['Loss', 'Validation Loss'])

In [27]:
history_results.to_csv('results/Simple_LSTM_60secs.csv')
model.save('models/Simple_LSTM_60secs')

INFO:tensorflow:Assets written to: models/Simple_LSTM_60secs\assets


INFO:tensorflow:Assets written to: models/Simple_LSTM_60secs\assets


In [28]:
!pip freeze

absl-py==1.0.0
anyio==3.5.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
attrs==21.4.0
Babel==2.9.1
backcall==0.2.0
black==22.1.0
bleach==4.1.0
cachetools==5.0.0
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.12
click==8.0.4
colorama==0.4.4
cycler==0.11.0
Cython==0.29.28
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
ephem==4.1.3
executing==0.8.2
flatbuffers==2.0
fonttools==4.29.1
gast==0.5.3
google-auth==2.6.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.44.0
h5py==3.6.0
idna==3.3
importlib-metadata==4.11.2
influxdb==5.3.1
ipykernel==6.9.1
ipython==8.0.1
ipython-genutils==0.2.0
jedi==0.18.1
Jinja2==3.0.3
joblib==1.1.0
json5==0.9.6
jsonschema==4.4.0
jupyter-client==7.1.2
jupyter-core==4.9.2
jupyter-server==1.13.5
jupyterlab==3.2.9
jupyterlab-pygments==0.1.2
jupyterlab-server==2.10.3
keras==2.8.0
Keras-Preprocessing==1.1.2
kiwisolver==1.3.2
korean-lunar-calendar==0.2.1
libclang==13.0.0
Markdown==3.3.6
Marku